In [3]:
! jt -t gruvboxd -T

In [ ]:
%load_ext pycodestyle_magic
!echo "111"

In [35]:
%%pycodestyle
a = 1


3:1: W391 blank line at end of file


In [3]:
%%pycodestyle
import math
import random
import torch
import numpy as np
import torch.nn.functional
import matplotlib.pyplot as plt
import torch.optim as opt
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import copy
import scipy.stats as st
from sklearn.datasets.samples_generator import make_blobs
from matplotlib.colors import LogNorm
import matplotlib.cm as cm

from scipy.interpolate import griddata

In [4]:
%%pycodestyle
# global veriable
Uniform_low_bound = 0
Uniform_up_bound = 1
Agent_number_n = 3

number_of_groups = 2

Normal_loc = 0.5
Normal_scale = 0.2
Normal_loc1 = 0.15
Normal_loc2 = 0.85
Normal_scale1 = 0.1
Normal_scale2 = 0.1
Distribution_number = 10000


16:1: W391 blank line at end of file


In [5]:
%%pycodestyle
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
print(dev)

7:1: W391 blank line at end of file


In [6]:
%%pycodestyle
seed = 7
np.random.seed(seed)
# exec(open('distribution/twopeak.py').read())
exec(open('../distribution/twopeak.py').read())
# exec(open('distribution/normal.py').read())
X_train,  X_test = train_test_split(value_list, 
                                    test_size=0.5, random_state=seed)

dataset_size = len(X_train)
print(dataset_size)
print(X_train[:100])
print(len(X_test))
# run_cs()


7:48: W291 trailing whitespace
15:1: W391 blank line at end of file


In [7]:
% % pycodestyle


def cost_sharing_with_deadline(test, t_c, target):
    temp_max_delay_list = torch.zeros(len(test)).cuda()
    temp_sum_delay = 0
    result = False
    for k in range(len(test), 0, -1):
        count = 0
        delay = 0
        for ii in range(len(test)):
            item = test[ii]
            if(item + 1e-4 >= target/k):
                count += 1
            else:
                delay += t_c[ii]
                temp_max_delay_list[ii] = t_c[ii]

        if(count >= k):
            temp_sum_delay += delay
            result = True  # 1.0#True
            break
        if(k <= 1):
            # print(test,number_n);
            temp_max_delay_list = t_c
            temp_sum_delay = torch.sum(t_c)
            result = False  # 0.0#False

    return temp_max_delay_list, temp_sum_delay, result

7:18: E703 statement ends with a semicolon
8:18: E703 statement ends with a semicolon
12:27: E703 statement ends with a semicolon
14:33: E703 statement ends with a semicolon
15:40: E225 missing whitespace around operator
17:17: E225 missing whitespace around operator
18:27: E225 missing whitespace around operator
18:34: E703 statement ends with a semicolon
19:19: E225 missing whitespace around operator
19:24: E261 at least two spaces before inline comment
19:24: E262 inline comment should start with '# '
20:18: E703 statement ends with a semicolon
21:13: E225 missing whitespace around operator
22:13: E265 block comment should start with '# '
23:32: E225 missing whitespace around operator
24:27: E225 missing whitespace around operator
24:42: E703 statement ends with a semicolon
25:19: E225 missing whitespace around operator
25:25: E261 at least two spaces before inline comment
25:25: E262 inline comment should start with '# '
27:31: E231 missing whitespace after ','
27:46: E231 missing 

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):

        super(Net, self).__init__()

        num_input = Agent_number_n
        num_hidden = 50
        num_output = Agent_number_n

        self.hidden_0 = torch.nn.Linear(num_input, num_hidden)
        self.hidden_1 = torch.nn.Linear(num_hidden, num_hidden)
        self.hidden_2 = torch.nn.Linear(num_hidden, num_hidden)
        self.output = torch.nn.Linear(num_hidden, num_output)
        self.deadline = torch.nn.Parameter(torch.ones(
            Agent_number_n, requires_grad=True), requires_grad=True).cuda()

    def forward(self, value_list):  # type_offender val_offender   ;  type_defender val_defender

        tensor = torch.ones(Agent_number_n, requires_grad=True)
        # 把鸡蛋放到篮子里, requires_grad是参不参与误差反向传播, 要不要计算梯度
        variable = Variable(tensor, requires_grad=True)

        h1 = torch.relu_(self.hidden_0(variable.cuda()))
        #h1 = torch.relu_(self.hidden_0(value_tensor))
        h2 = torch.relu_(self.hidden_1(h1))
        h3 = torch.relu_(self.hidden_2(h2))
        h4 = self.output(h3)       # no relu!!!!
        # torch.sigmoid(self.deadline)# need more layer
        deadline = torch.sigmoid((h4+1)*10)
        #deadline = torch.sigmoid(self.deadline)
        # print(deadline)
        # return

        deadline_i_list = []
        for i in range(Agent_number_n):
            deadline_i = []
            for j in range(Agent_number_n):
                if(i != j):
                    deadline_i.append(float(deadline[j]))
            deadline_i_list.append(deadline_i)
        deadline_i_list = torch.Tensor(deadline_i_list)

        lack_matrix = [[] for k in range(Agent_number_n)]  # k means lack of k
        for k in range(len(value_list)):
            test = value_list[k]
            for i in range(Agent_number_n):
                test_i = []
                for j in range(Agent_number_n):
                    if(i != j):
                        test_i.append(float(test[j] * deadline[j]))
                lack_matrix[i].append(test_i)

        temp_sum_delay_total = torch.zeros(1).cuda()
        temp_max_delay_total = torch.zeros(1).cuda()

#         print(deadline)
#         for i in range(Agent_number_n):
#              print(deadline[i].requires_grad)
#         return

        for k in range(len(value_list)):
            value_tensor = value_list[k]

            # print(value_tensor.cpu())

            test = copy.deepcopy(value_tensor)
            #test_change = copy.deepcopy(X_test[i]);
            test_change = []

            for i in range(len(test)):
                test_change.append(test[i] * deadline[i])

            temp_max_delay_list, temp_sum_delay, judge1 = cost_sharing_with_deadline(test_change,
                                                                                     deadline, 1.0)
            temp_max_delay_list = temp_max_delay_list.cuda()

            temp_max_delay_list_copy = torch.zeros(
                Agent_number_n, requires_grad=True)
            for i in range(Agent_number_n):
                temp_max_delay_list_copy[i] = temp_max_delay_list_copy[i] + \
                    temp_max_delay_list[i]

            for i in range(Agent_number_n):
                Possible = 0
                deadline_i = deadline_i_list[i]
                for j in range(len(lack_matrix[i])):
                    test_i = lack_matrix[i][j]
                    temp_max_delay_i, temp_sum_delay_i, judge_i = cost_sharing_with_deadline(test_i,
                                                                                             deadline_i, 1.0)
                    if(judge_i == False):
                        Possible = Possible + 1.0
                temp = (1.0-deadline[i].clone()) * \
                    torch.tensor(Possible/len(lack_matrix[i])).cuda()
                temp_sum_delay = temp_sum_delay + temp

#                 print(k,i)
#                 print(deadline[i],deadline[i].requires_grad)
#                 print(deadline[i].clone(),deadline[i].clone().requires_grad)
#                 print(temp_max_delay_list[i],temp_max_delay_list[i].requires_grad)
#                 print(temp,temp.requires_grad)
#                 print()
                temp_max_delay_list_copy[i] = temp_max_delay_list_copy[i] + temp

            temp_sum_delay_total = temp_sum_delay_total + temp_sum_delay
            temp_max_delay_total = temp_max_delay_total + \
                torch.max(temp_max_delay_list)
#         for i in range(Agent_number_n):
#             print(temp_max_delay_list[i].requires_grad)

#         return

        return temp_max_delay_total, temp_sum_delay_total, deadline.cpu().data.numpy(), float(temp_sum_delay)

In [ ]:
def weight_init(m):
    if isinstance(m, torch.nn.Conv2d):
        torch.nn.init.xavier_normal_(m.weight, gain=nn.init.calculate_gain('relu'))
        torch.nn.init.zeros_(m.bias)
    elif isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)
        

In [ ]:
random.seed(2000)
torch.manual_seed(256)
net = Net()
# net.apply(weight_init)

In [ ]:
#net = torch.load("Deep_learning_with_deadline_5")


In [ ]:
net.to(dev)

In [ ]:
#optimizer = opt.RMSprop(net.parameters(), lr=0.00001)
#optimizer = opt.SGD(net.parameters(), lr=0.002)
optimizer = opt.Adam(net.parameters(), lr=0.00005)

batch_size = 32
echo = 501

In [ ]:
for i in range(int(echo)):

    # offender_types = []
    # defender_types = []
    loss_sum = 0
    denominator = 0
    """
    for j in range(batch_size):
        offender_types.append(random.randint(0, 400))
        defender_types.append(random.randint(0, 15))
    """
    X_train_list=[]
    for j in range(batch_size):
        index_random = random.randint(0,len(X_train)-1)
        X_train_list.append(torch.from_numpy(X_train[index_random]).cuda().type(torch.float32))
        denominator+=1
        
    h_delay_max,h_delay_sum,deadline_R,delay_R   = net(X_train_list)
    loss_sum += h_delay_sum
        
        

    optimizer.zero_grad()
    #loss = torch.square(loss_function(loss_sum / denominator) + 52)
    loss = loss_sum / denominator
    
    loss.backward()
    optimizer.step()
    
    if i % 50 == 0:
        print("i",i)
        print("batch_loss: %.2f " % float(loss_sum/denominator))
        print("value:" , X_train[index_random])
        print("deadline:" , deadline_R)
        #print("delay:" , delay_R)
        print()
    

In [ ]:
print("batch: %.2f " % float(loss_sum/denominator))
print("value:" , X_train[index_random])
print("deadline:" , deadline_R)
print("delay:" , delay_R)
    

In [ ]:
#torch.save(net, "Deep_learning_with_deadline_5")

In [ ]:
def cost_sharing_with_deadline_old(test,t_c,target):
    temp_max_delay_list=[0 for i in range(len(test))]
    temp_sum_delay=0
    result=False
    for k in range(len(test),0,-1):
        count=0;
        delay=0;
        for ii in range(len(test)):
            item= test[ii]
            if(item+1e-4>=target/k):
                count+=1;
            else:
                delay+=t_c[ii];
                temp_max_delay_list[ii]=t_c[ii]
            
        if(count>=k):
            temp_sum_delay+=delay;
            result=True
            break;
        if(k<=1):
            #print(test,number_n);
            temp_max_delay_list=t_c
            temp_sum_delay=sum(t_c);
            result=False
            
    return temp_max_delay_list,temp_sum_delay,result



#Cost Sharing
def run_cs_old(deadline_list):
    sum_delay=0
    max_delay=0
    test_number=0
    for i in range(len(X_test)):
        test_number+=1
        temp_max_delay=0
        temp_delay=0
        test = copy.deepcopy(X_test[i])
        #test_change = copy.deepcopy(X_test[i]);
        test_change = []

        for j in range(len(test)):
            test_change.append(test[j] * deadline_list[j])
            
        
        temp_max_delay_list,temp_sum_delay,judge1 = cost_sharing_with_deadline_old(test_change,
                        copy.deepcopy(deadline_list),1.0)
        for j in range(len(test_change)):
            test_i = copy.deepcopy(test_change);
            test_i = np.delete(test_i, j)
            
            deadline_i = copy.deepcopy(deadline_list);
            deadline_i = np.delete(deadline_i, j)
            
            temp_max_delay_i_list,temp_sum_delay_i,judge_i = cost_sharing_with_deadline_old(test_i,
                        deadline_i,1.0)
            
            if(judge_i==False):
                temp_sum_delay += (1.0-deadline_list[j])
                temp_max_delay_list[j] += (1.0-deadline_list[j])
                
            
        
        max_delay+=max(temp_max_delay_list)
        
        sum_delay+=temp_sum_delay
        
    print("deadline: ",deadline_list)
    print("sum_delay: ",sum_delay/test_number)
    print("max_delay: ",max_delay/test_number)
    print()
    return max_delay/test_number,sum_delay/test_number
    
print(run_cs_old(deadline_R))


In [ ]:
#torch.save(net, "Deep_learning_with_deadline_n=3")